# Building the amount matrix

In [1]:
import json
import pandas as pd
with open('./train.json', 'r') as f:
    recipes = json.load(f)

df = pd.read_csv('./ingredients_refined.csv')

In [12]:
matrix = pd.DataFrame(columns=df['ingredient'].values)

In [18]:
matrix

,lemon,cream,yogurt,vanilla,shrimp,cilantro,jalapeno,parsley,vinegar,tortillas,...,olives,onions,peppers,pork loin,lettuce,garlic,onion,cheese,chicken breast,pork chops


In [22]:
recipe_number = 1
for recipe in recipes:
    ingredients = recipe['ingredients']
    for ingredient in ingredients:
        matrix[ingredient].iloc[recipe_number] = 1
    matrix = matrix.fillna(0)
    recipe_number += 1

/var/folders/l7/ywb8q5r53j33j1lqzbzjyshm0000gn/T/ipykernel_31755/101412243.py:5: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  matrix[ingredient].iloc[recipe_number] = 1


IndexError: iloc cannot enlarge its target object

# Modeling

In [3]:
import pandas as pd
import numpy as np 
df = pd.read_csv('./ingredients_refined.csv')
ingredients = np.array(df['ingredient'].values)
price = df['price']
metric = df['metric']

In [4]:
import gamspy as gp
import numpy as np

cont = gp.Container()
I = cont.addSet('I', records = range(1, len(ingredients) + 1))
R = cont.addSet('R', records = range(1, len(recipes) + 1))